In [1]:
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import h5py
import numba as nb
from numba import jit, njit
import tracerFuncs as tF
import insituFuncs as iF

plt.style.use('default')
plt.rcParams["figure.figsize"][0] = 16
plt.rcParams["figure.figsize"][1] = 9
plt.rcParams['errorbar.capsize']=2

In [2]:
basePath='/virgotng/universe/IllustrisTNG/TNG50-2/output'
h_const=il.groupcat.loadHeader(basePath,99)['HubbleParam']
boxSize = il.groupcat.loadHeader(basePath,99)['BoxSize']
print(h_const)

0.6774


do traceback with different redshifts:

In [5]:
%%time
tF.TraceBackAllInsituStars(basePath,99,25)

CPU times: user 22min 24s, sys: 1min 8s, total: 23min 33s
Wall time: 24min 37s


# test:

In [48]:
all_target_gas_ids = il.snapshot.loadSubset(basePath,99,0,['ParticleIDs'])
all_target_star_ids = il.snapshot.loadSubset(basePath,99,4,['ParticleIDs'])

In [52]:
def match(ar1, ar2, is_sorted = False):
    """ Returns index arrays i1,i2 of the matching elements between ar1 and ar2. While the elements of ar1 
        must be unique, the elements of ar2 need not be. For every matched element of ar2, the return i1 
        gives the index in ar1 where it can be found. For every matched element of ar1, the return i2 gives 
        the index in ar2 where it can be found. Therefore, ar1[i1] = ar2[i2]. The order of ar2[i2] preserves 
        the order of ar2. Therefore, if all elements of ar2 are in ar1 (e.g. ar1=all TracerIDs in snap, 
        ar2=set of TracerIDs to locate) then ar2[i2] = ar2. The approach is one sort of ar1 followed by 
        bisection search for each element of ar2, therefore O(N_ar1*log(N_ar1) + N_ar2*log(N_ar1)) ~= 
        O(N_ar1*log(N_ar1)) complexity so long as N_ar2 << N_ar1. """
    
    if not is_sorted:
        # need a sorted copy of ar1 to run bisection against
        index = np.argsort(ar1)
        ar1_sorted = ar1[index]
    else:
        ar1_sorted = ar1
        index = np.arange(ar1.shape[0])
    # NlogN search of ar1_sorted for each element in ar2
    ar1_sorted_index = np.searchsorted(ar1_sorted, ar2)

    # undo sort
    ar1_inds = np.take(index, ar1_sorted_index, mode="clip")

    # filter out non-matches
    mask = (ar1[ar1_inds] == ar2)
    ar2_inds = np.where(mask)[0]
    
    #fill non-matches with -1
    ar1_inds[np.where(np.logical_not(mask))[0]] = -1
    #ar1_inds = ar1_inds[ar2_inds]

    return ar1_inds, ar2_inds

In [66]:
def parentIndicesOfAll(parent_ids, all_gas_ids, all_star_ids): 
    #which parent corresponds to which gas/star particles?
    #for this: save index into gas('0') / star('1') subset array
    target_parent_indices = np.zeros((len(parent_ids),2))
    gas_inds, _ = match(all_gas_ids,parent_ids) #only inds1 relevant
    star_inds, _ = match(all_star_ids,parent_ids) #only inds1 relevant
    
    #gas_inds now contains either the indices of gas parents into the gas subset or -1; same for star_inds
    target_parent_indices[:,0] = gas_inds
    target_parent_indices[:,1] = 0
    target_parent_indices[np.where(gas_inds == -1)[0],1] = 1 #no gas index found => parent is a star
    target_parent_indices[np.where(gas_inds == -1)[0],0] = star_inds[np.where(star_inds!=-1)[0]]
    del gas_inds, star_inds
    return target_parent_indices

In [72]:
%%time
a = parentIndicesOfAll(all_target_gas_ids,all_target_gas_ids,all_target_star_ids)

CPU times: user 1min 29s, sys: 1.74 s, total: 1min 31s
Wall time: 1min 31s


In [74]:
test = np.where(a[:,1]==1)[0]
print(test)

[]


In [53]:
ar2 = np.array([1,1,4,3,0,4]) #parent_ids
ar1 = np.array([4,1,0]) #gas_ids
inds1, inds2 = match(ar1,ar2)

In [54]:
print(inds1)
print(inds2)

[ 1  1  0 -1  2  0]
[0 1 2 4 5]
